In [1]:
# Imports

import pandas as pd
import numpy as np
import time

In [2]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_size, state=None):
        if state is None:
            # Inital board set up
            self.state = np.random.randint(0, board_size, board_size)
        else:
            self.state = np.array(state) 

        # Set the goal fitness value where there is no queen attacks 
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
    
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Count of the queen attacks on the row 
            count += (current_queen == remaining_queens).sum()
            
            # Count of the queen attacks on the diagonal 
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            # Total queen on queen attacks
            count += sum((remaining_queens == upper_diagonal) | (remaining_queens == lower_diagonal))

        # Return the final fitness 
        return self.goal_fitness - count

In [3]:
# Genetic Algorithm

class GeneticAlgorithm:
    def __init__(self, board_size, population_size, mutation_probability, max_generations, initial_population=None):
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.max_generations = max_generations
        self.last_state = None

        # Set the goal fitness value where there is no queen attacks
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        
        if initial_population is None:
            # Inital population set up 
            self.current_population = [EightQueensState(board_size) for _ in range(population_size)]
        else:
            self.current_population = initial_population


    def calculate_population_fitness(self):
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()

            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Calculate selection probability
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness
            
        # Sort the population by the highest selection probabilites 
        return sorted(population_data, key=lambda x: x['selection_prob'], reverse=True)


    def select_parent(self, population_data):
        probabilities = [individual['selection_prob'] for individual in population_data]
        idx = np.random.choice(len(population_data), p=probabilities)
        return population_data[idx]['state']


    def crossover(self, parent1, parent2):
        idx = np.random.randint(1, len(parent1))
        return np.concatenate([parent1[:idx], parent2[idx:]]), np.concatenate([parent2[:idx], parent1[idx:]])


    def mutate(self, state):
        if np.random.random() < self.mutation_probability:
            idx = np.random.randint(len(state))
            state[idx] = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
        return state


    def create_next_generation(self, population_data):
        next_generation = []
        
        # Add elitism - keeping the best individual
        next_generation.append(EightQueensState(self.board_size, population_data[0]['state']))
        
        while len(next_generation) < self.population_size:
            parent1 = self.select_parent(population_data)
            parent2 = self.select_parent(population_data)
            
            child1, child2 = self.crossover(parent1, parent2)
            
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        # Trim if we added one too many children
        self.current_population = next_generation[:self.population_size]


    def run(self):
        for generation in range(self.max_generations):
            population_data = self.calculate_population_fitness()
            
            # Display chess board solution if the fitness of the individual matches the goal
            if population_data[0]['fitness'] == self.goal_fitness:
                self.last_state = population_data[0]['state'] 
                print(self.goal_fitness, population_data[0]['fitness'])
                print(f"Solution found in generation {generation}")
                self.display_chess()
                return
            
            self.create_next_generation(population_data)
    

    def display_chess(self):
        for row in range(self.board_size):
            print(f'{8-row} |', end=' ')
            for col in range(self.board_size):
                if self.last_state[col] == row:
                    print('Q ', end='')  
                else:
                    print('- ', end='')  
            print(f'|')
    

In [5]:
# Run

def run_multiple_trials(trails):
    for trial in range(trails):
        print(f"\nTrial {trial + 1}:")
        
        start_time = time.time()
        
        ga = GeneticAlgorithm(
            board_size=8,
            population_size=300,
            mutation_probability=0.15,
            max_generations=1000000
        )
        
        ga.run()
        
        end_time = time.time()
        time_taken = end_time - start_time
        
        print(f"Time taken: {time_taken:.2f} seconds")


run_multiple_trials(50)


Trial 1:
28 28
Solution found in generation 69
8 | - - - - - - Q - |
7 | - - Q - - - - - |
6 | Q - - - - - - - |
5 | - - - - - Q - - |
4 | - - - - - - - Q |
3 | - - - - Q - - - |
2 | - Q - - - - - - |
1 | - - - Q - - - - |
Time taken: 1.15 seconds

Trial 2:
28 28
Solution found in generation 71
8 | - - - - - Q - - |
7 | - - Q - - - - - |
6 | - - - - - - Q - |
5 | - Q - - - - - - |
4 | - - - - - - - Q |
3 | - - - - Q - - - |
2 | Q - - - - - - - |
1 | - - - Q - - - - |
Time taken: 1.15 seconds

Trial 3:
28 28
Solution found in generation 212
8 | - - - - - - Q - |
7 | Q - - - - - - - |
6 | - - Q - - - - - |
5 | - - - - - - - Q |
4 | - - - - - Q - - |
3 | - - - Q - - - - |
2 | - Q - - - - - - |
1 | - - - - Q - - - |
Time taken: 3.50 seconds

Trial 4:
28 28
Solution found in generation 72
8 | - - - - Q - - - |
7 | - - Q - - - - - |
6 | Q - - - - - - - |
5 | - - - - - - Q - |
4 | - Q - - - - - - |
3 | - - - - - - - Q |
2 | - - - - - Q - - |
1 | - - - Q - - - - |
Time taken: 1.23 seconds

Tr

In [ ]:
# Plot GA

def plot_ga(solution, fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(0, 255, 0, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Final Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        height = 700,
        width=1500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [ ]:
# Optimisation

import numpy as np
from tqdm import tqdm
import time
from dataclasses import dataclass
import contextlib
import io

@dataclass
class OptimizationResult:
    pop_size: int
    mut_prob: float
    generations: int
    time_taken: float

def test_parameters(params) -> OptimizationResult:
    """Run GA with given parameters and return generations needed to find solution"""
    board_size, pop_size, mut_prob = params
    start_time = time.time()
    
    ga = QuietGeneticAlgorithm(
        board_size=board_size,
        population_size=pop_size,
        mutation_probability=mut_prob,
        max_generations=1000
    )
    
    fitness_history, _ = ga.run()
    time_taken = time.time() - start_time
    
    # Find generation where solution was found
    for gen, fitness in enumerate(fitness_history):
        if fitness == ga.goal_fitness:
            return OptimizationResult(pop_size, mut_prob, gen, time_taken)
            
    return OptimizationResult(pop_size, mut_prob, 1000, time_taken)

def optimize_parameters(board_size=8, trials=10):  # Increased default trials
    """Test different parameter combinations prioritizing consistent performance"""
    # Expanded parameter ranges
    population_sizes = [100, 150, 200, 250, 300, 400, 500]  # Increased minimum population size
    mutation_probs = [0.05, 0.08, 0.1, 0.12, 0.15, 0.18]
    
    results = []
    
    # Create parameter combinations
    params = [(board_size, pop, mut) 
             for pop in population_sizes 
             for mut in mutation_probs]
    
    # Run trials with progress bar
    total_combinations = len(params) * trials
    with tqdm(total=total_combinations, desc="Testing combinations") as pbar:
        for _ in range(trials):
            for param_set in params:
                result = test_parameters(param_set)
                results.append(result)
                pbar.update(1)
    
    # Process results
    performance = {}
    for result in results:
        key = (result.pop_size, result.mut_prob)
        if key not in performance:
            performance[key] = {'gens': [], 'times': []}
        performance[key]['gens'].append(result.generations)
        performance[key]['times'].append(result.time_taken)
    
    # Calculate performance metrics with emphasis on consistency
    avg_performance = {}
    for k, v in performance.items():
        gen_array = np.array(v['gens'])
        time_array = np.array(v['times'])
        success_mask = gen_array < 1000
        
        if sum(success_mask) > 0:  # Only consider combinations with some successes
            successful_gens = gen_array[success_mask]
            successful_times = time_array[success_mask]
            
            # Calculate metrics
            success_rate = sum(success_mask) / len(gen_array)
            avg_gens = np.mean(successful_gens)
            std_gens = np.std(successful_gens)
            cv_gens = std_gens / avg_gens if avg_gens > 0 else float('inf')  # Coefficient of variation
            avg_time = np.mean(successful_times)
            max_time = np.max(successful_times)
            
            # Consistency score (lower is better)
            consistency_score = (cv_gens * 0.4 +                      # Weight variation heavily
                               (1 - success_rate) * 0.3 +            # Weight success rate
                               (max_time / 30) * 0.3)                # Penalize any runs over 30s
            
            avg_performance[k] = {
                'avg_gens': avg_gens,
                'std_gens': std_gens,
                'cv_gens': cv_gens,
                'success_rate': success_rate,
                'avg_time': avg_time,
                'max_time': max_time,
                'consistency_score': consistency_score
            }
    
    # Sort by consistency score (lower is better)
    sorted_params = sorted(avg_performance.items(), 
                         key=lambda x: x[1]['consistency_score'])
    
    return sorted_params

# Run optimization
print("Optimizing parameters...")
best_params = optimize_parameters(trials=10)  # Increased trials for better statistics

print("\nTop 5 Parameter Combinations:")
print("Pop Size | Mut Prob | Avg Gens | CV Gens | Success % | Avg Time | Max Time | Consistency")
print("-" * 95)
for (pop_size, mut_prob), metrics in best_params[:5]:
    print(f"{pop_size:^8d} | {mut_prob:^8.2f} | {metrics['avg_gens']:^8.1f} | "
          f"{metrics['cv_gens']:^7.2f} | {metrics['success_rate']*100:^8.1f}% | "
          f"{metrics['avg_time']:^8.2f}s | {metrics['max_time']:^8.2f}s | "
          f"{metrics['consistency_score']:^10.3f}")

# Print detailed recommendation
best_pop, best_mut = best_params[0][0]
best_metrics = best_params[0][1]
print("\nRecommended Parameters:")
print(f"Population Size: {best_pop}")
print(f"Mutation Probability: {best_mut}")
print(f"Expected Performance:")
print(f"- Success Rate: {best_metrics['success_rate']*100:.1f}%")
print(f"- Average Generations: {best_metrics['avg_gens']:.1f}")
print(f"- Coefficient of Variation: {best_metrics['cv_gens']:.2f}")
print(f"- Average Time: {best_metrics['avg_time']:.2f}s")
print(f"- Maximum Time: {best_metrics['max_time']:.2f}s")